In [15]:
import urllib.request
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions

from keras.layers import AveragePooling2D
from keras.preprocessing import image
from keras.models import Model
import numpy as np
import socket
socket.setdefaulttimeout(0.3)

In [2]:
base_model = InceptionV3(
    weights='imagenet',
    include_top=True
)

# We'll extract features at the final pool layer.
model = Model(
    inputs=base_model.input,
    outputs=base_model.get_layer('avg_pool').output
)

In [3]:
def extract_image(image_path,model):
        img = image.load_img(image_path, target_size=(299, 299))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        features = model.predict(x)
        return features[0]

In [4]:
import json

with open('train.json') as json_data:
    d = json.load(json_data)

In [5]:
import pickle

In [6]:
print("Number of elements in dictionary",len(d['images']))

Number of elements in dictionary 1014544


In [7]:
#extract image features
batch_size = 100
num_batches = len(d['images'])/batch_size
print(num_batches)
num_batches = 1
for i in range(num_batches):
    batch_list = []
    exception_count = 0
    for obj in d['images'][i*batch_size:i*batch_size+batch_size]:
        try:
            urllib.request.urlretrieve(obj['url'],'temp.jpg')
            features = extract_image('temp.jpg',model)
            batch_list.append((obj['imageId'],features))
        except Exception as e:
            exception_count += 1
            batch_list.append((obj['imageId'],np.random.rand(2048)))    
    print("Batch number was",i,"Exception count",exception_count)
    output = open('processed_train/train_'+str(i)+'.pkl', 'wb')
    pickle.dump(batch_list,output)

10145.44
Batch number was 0 Exception count 11


In [8]:
with open('test.json') as json_data:
    d_test = json.load(json_data)

In [9]:
label_mapping = {}
for obj in d['annotations']:
    label_mapping[obj['imageId']] = obj['labelId']

In [13]:
i = 68

In [16]:
#extract test image features
batch_size = 100
num_batches = int(len(d_test['images'])/batch_size)
print(num_batches)
while(1):
    batch_list = []
    exception_count = 0
    for obj in d['images'][i*batch_size:i*batch_size+batch_size]:
        try:
            urllib.request.urlretrieve(obj['url'],'temp.jpg',)
            features = extract_image('temp.jpg',model)
            batch_list.append((obj['imageId'],features))
        except Exception as e:
            exception_count += 1
            batch_list.append((obj['imageId'],np.random.rand(2048)))    
    print("Batch number was",i,"Exception count",exception_count)
    output = open('processed_test/test_'+str(i)+'.pkl', 'wb')
    pickle.dump(batch_list,output)
    i = i + 1
    if i>(num_batches-1):
        break

397
Batch number was 68 Exception count 29
Batch number was 69 Exception count 80
Batch number was 70 Exception count 32
Batch number was 71 Exception count 91
Batch number was 72 Exception count 88
Batch number was 73 Exception count 72
Batch number was 74 Exception count 64
Batch number was 75 Exception count 70
Batch number was 76 Exception count 71
Batch number was 77 Exception count 95
Batch number was 78 Exception count 88
Batch number was 79 Exception count 87
Batch number was 80 Exception count 88
Batch number was 81 Exception count 95
Batch number was 82 Exception count 85
Batch number was 83 Exception count 51
Batch number was 84 Exception count 24
Batch number was 85 Exception count 61
Batch number was 86 Exception count 77
Batch number was 87 Exception count 81
Batch number was 88 Exception count 77
Batch number was 89 Exception count 79
Batch number was 90 Exception count 82
Batch number was 91 Exception count 85
Batch number was 92 Exception count 82
Batch number was 93 E

KeyboardInterrupt: 

In [17]:
len(d_test['images'])

39706